<a href="https://colab.research.google.com/github/xogmlkim/re/blob/main/%EA%B9%80%ED%83%9C%ED%9D%AC_cats_and_dogs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1.CNN
- 목표 val loss: 0.24087

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.callbacks import ModelCheckpoint
import os

# import dataset
dataset_name = 'cats_vs_dogs'
setattr(tfds.image_classification.cats_vs_dogs, '_URL',"https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_5340.zip")
train_dataset, info = tfds.load(name=dataset_name, split='train[:80%]', with_info=True)
valid_dataset, info = tfds.load(name=dataset_name, split='train[-20%:]', with_info=True)

# data preprocessing
def normalize(images):
    img, lbl = tf.cast(images['image'], tf.float32) / 255.0, images['label']
    img = tf.image.resize(img, size=(224, 224))
    lbl = tf.one_hot(lbl, depth=2)
    return img, lbl

batch_size = 32
train_data = train_dataset.map(normalize).batch(batch_size)
valid_data = valid_dataset.map(normalize).batch(batch_size)

# model building
model = tf.keras.models.Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3), padding='same'),
    MaxPooling2D(2, 2),
    Dropout(0.3),
    Conv2D(64, (3, 3), activation='relu', padding='same'),
    MaxPooling2D(2, 2),
    Dropout(0.3),
    Conv2D(64, (3, 3), activation='relu', padding='same'),
    MaxPooling2D(2, 2),
    Dropout(0.3),
    Conv2D(64, (3, 3), activation='relu', padding='same'),
    MaxPooling2D(2, 2),
    Dropout(0.3),
    Flatten(),
    
    Dense(64, activation='relu'),
    Dense(2, activation='softmax')])     

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# model monitoring
checkpoint = ModelCheckpoint("best_model.h5", monitor='val_loss', verbose=1, save_best_only=True, mode='min', save_weights_only=True)

# model training
history = model.fit(train_data, epochs=5, validation_data=valid_data, callbacks=[checkpoint], verbose=1)

# save
model.save("mymodel_cats_and_dogs.h5")


Epoch 1/5
582/582 [==============================] - ETA: 0s - loss: 0.6485 - accuracy: 0.6109
Epoch 1: val_loss improved from inf to 0.55543, saving model to best_model.h5
582/582 [==============================] - 51s 83ms/step - loss: 0.6485 - accuracy: 0.6109 - val_loss: 0.5554 - val_accuracy: 0.7315
Epoch 2/5
581/582 [============================>.] - ETA: 0s - loss: 0.5437 - accuracy: 0.7204
Epoch 2: val_loss improved from 0.55543 to 0.51255, saving model to best_model.h5
582/582 [==============================] - 46s 80ms/step - loss: 0.5436 - accuracy: 0.7204 - val_loss: 0.5126 - val_accuracy: 0.7496
Epoch 3/5
581/582 [============================>.] - ETA: 0s - loss: 0.4875 - accuracy: 0.7633
Epoch 3: val_loss improved from 0.51255 to 0.47451, saving model to best_model.h5
582/582 [==============================] - 46s 79ms/step - loss: 0.4876 - accuracy: 0.7631 - val_loss: 0.4745 - val_accuracy: 0.7732
Epoch 4/5
581/582 [============================>.] - ETA: 0s - loss: 0.444

# 2.Transfer learning

In [ ]:
import tensorflow_datasets as tfds
import tensorflow as tf
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.applications import VGG16

from IPython.display import Image
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# for Transfer learning
from tensorflow.keras.applications import VGG16

# import dataset
BATCH_SIZE = 32
EPOCHS = 5
dataset_name = 'cats_vs_dogs'
setattr(tfds.image_classification.cats_vs_dogs, '_URL',"https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_5340.zip")
(train_dataset, validation_dataset), info = tfds.load(name=dataset_name, split=('train[:80%]', 'train[-20%:]'), with_info=True)

# data preprocessing
def normalize(images):
    # normalize
    img, lbl = tf.cast(images['image'], tf.float32) / 255.0, images['label']
    # resize
    img = tf.image.resize(img, size=(224, 224))
    return img, lbl

train_data = train_dataset.map(normalize).batch(BATCH_SIZE)
valid_data = validation_dataset.map(normalize).batch(BATCH_SIZE)


transfer_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
# Freeze all layers in the VGG16 model so they are not updated during training
for layer in transfer_model.layers:
    layer.trainable = False

# model building
model_cnn = Sequential([
    transfer_model,
    Flatten(),
    Dropout(0.2),
    Dense(64, activation='relu'), 
    Dense(32, activation='relu'), 
    Dense(1, activation='sigmoid')])
model_cnn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


# model monitoring
early_stopping = EarlyStopping(monitor='val_loss', patience=10)
checkpoint_path = 'checkpoint.ckpt'
checkpoint = ModelCheckpoint(checkpoint_path, 
        save_best_only=True, 
        save_weights_only=True, 
        monitor='val_loss', 
        verbose=1)

# model training
cnn_history = model_cnn.fit(train_data, validation_data=(valid_data), epochs=5, callbacks=[checkpoint, early_stopping])
model_cnn.load_weights(checkpoint_path)                      
model_cnn.save("cats-dogs-cnn.h5")      


Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/cats_vs_dogs/4.0.0.incomplete8XCUQF/cats_vs_dogs-train.tfrecord*...:   0%|…

Dataset cats_vs_dogs downloaded and prepared to /root/tensorflow_datasets/cats_vs_dogs/4.0.0. Subsequent calls will reuse this data.
58889256/58889256 [==============================] - 3s 0us/step
Epoch 1/5
582/582 [==============================] - ETA: 0s - loss: 0.2290 - accuracy: 0.8999
Epoch 1: val_loss improved from inf to 0.17438, saving model to checkpoint.ckpt
582/582 [==============================] - 105s 157ms/step - loss: 0.2290 - accuracy: 0.8999 - val_loss: 0.1744 - val_accuracy: 0.9256
Epoch 2/5
582/582 [==============================] - ETA: 0s - loss: 0.1465 - accuracy: 0.9411
Epoch 2: val_loss improved from 0.17438 to 0.15959, saving model to checkpoint.ckpt
582/582 [==============================] - 93s 160ms/step - loss: 0.1465 - accuracy: 0.9411 - val_loss: 0.1596 - val_accuracy: 0.9321
Epoch 3/5
582/582 [==============================] - ETA: 0s - loss: 0.1252 - accuracy: 0.9494
Epoch 3: val_loss did not improve from 0.15959
582/582 [============================

# 3.Chat GPT
**prompt**

Please code fast and efficient classification model from cats and dogs dataset from tensorflow. Here is some requirments and purpose to make the model.
- Only Import dataset from below the link. -> 그러나 이 부분에서 데이터셋 오류 발생 
https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_5340.zip
- Use any method to make lowest val_loss.
- Use ImageData Generator.
- Use checkpoint with save best only, and save weight only.
- Set verbose = 1
- Save the model with best performance.

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.callbacks import ModelCheckpoint
import os


# import dataset
dataset_name = 'cats_vs_dogs'
setattr(tfds.image_classification.cats_vs_dogs, '_URL',"https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_5340.zip")
train_dataset, info = tfds.load(name=dataset_name, split='train[:80%]', with_info=True)
valid_dataset, info = tfds.load(name=dataset_name, split='train[-20%:]', with_info=True)

# data preprocessing
def normalize(images):
    img, lbl = tf.cast(images['image'], tf.float32) / 255.0, images['label'] 
    img = tf.image.resize(img, size=(224, 224))
    return img, lbl

batch_size = 32
train_data = train_dataset.map(normalize).batch(batch_size)
valid_data = valid_dataset.map(normalize).batch(batch_size)

# Build the convolutional neural network model
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(224, 224, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# Compile the model with binary cross-entropy loss and Adam optimizer
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Set up checkpoint callback to save the best model weights
checkpoint = ModelCheckpoint("best_model.h5", monitor='val_loss', verbose=1, save_best_only=True, mode='min', save_weights_only=True)

# Train the model with fit
history = model.fit(train_data, epochs=5, validation_data=valid_data, callbacks=[checkpoint], verbose=1)

# Save the final model
model.save("mymodel_cats_and_dogs.h5")


Epoch 1/5
582/582 [==============================] - ETA: 0s - loss: 0.6759 - accuracy: 0.5785
Epoch 1: val_loss improved from inf to 0.62162, saving model to best_model.h5
582/582 [==============================] - 26s 42ms/step - loss: 0.6759 - accuracy: 0.5785 - val_loss: 0.6216 - val_accuracy: 0.6610
Epoch 2/5
582/582 [==============================] - ETA: 0s - loss: 0.5997 - accuracy: 0.6805
Epoch 2: val_loss improved from 0.62162 to 0.58388, saving model to best_model.h5
582/582 [==============================] - 25s 43ms/step - loss: 0.5997 - accuracy: 0.6805 - val_loss: 0.5839 - val_accuracy: 0.7042
Epoch 3/5
582/582 [==============================] - ETA: 0s - loss: 0.4979 - accuracy: 0.7558
Epoch 3: val_loss improved from 0.58388 to 0.46610, saving model to best_model.h5
582/582 [==============================] - 25s 42ms/step - loss: 0.4979 - accuracy: 0.7558 - val_loss: 0.4661 - val_accuracy: 0.7814
Epoch 4/5
581/582 [============================>.] - ETA: 0s - loss: 0.391